In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import pickle
import os
import gc

# ✅ 설정
CAFE_URL = "https://cafe.naver.com/f-e/cafes/19997929/menus/0?q=%EC%8B%9C%ED%97%98&ta=SUBJECT&page=1&size=50"
COOKIE_PATH = "naver_cookies.pkl"

# ✅ Chrome 옵션 설정
options = Options()
options.add_argument("start-maximized")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# ✅ WebDriver 실행
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# ✅ 봇 감지 방지용 JavaScript 주입
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": "Object.defineProperty(navigator, 'webdriver', { get: () => undefined })"
})

# ✅ 수동 로그인 후 쿠키 저장
def manual_login_and_save_cookie():
    driver.get("https://nid.naver.com/nidlogin.login")
    print("🔐 수동 로그인 해주세요 (60초 대기)")
    time.sleep(60)
    pickle.dump(driver.get_cookies(), open(COOKIE_PATH, "wb"))
    print("✅ 쿠키 저장 완료")

# ✅ 쿠키 불러와 로그인 유지
def load_cookie_and_login():
    driver.get("https://www.naver.com")
    cookies = pickle.load(open(COOKIE_PATH, "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.get("https://cafe.naver.com")
    for cookie in cookies:
        try:
            driver.add_cookie(cookie)
        except:
            pass
    driver.get(CAFE_URL)

# ✅ 게시글 목록 추출
def get_article_elements():
    tbodys = driver.find_elements(By.TAG_NAME, "tbody")
    if not tbodys:
        return []
    last_tbody = tbodys[-1]
    return last_tbody.find_elements(By.CSS_SELECTOR, "a.article")

# ✅ 게시글 상세 내용 추출 (여러 댓글 포함)
def extract_post_data():
    wait = WebDriverWait(driver, 10)
    title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".title_text"))).text
    nickname = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".nickname"))).text
    date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.date"))).text
    content = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.se-main-container"))).text
    try:
        comment_elements = driver.find_elements(By.CSS_SELECTOR, "span.text_comment")
        comments = [c.text.strip() for c in comment_elements if c.text.strip()]
        comment_text = " || ".join(comments) if comments else None
    except:
        comment_text = None
    return {
        "제목": title,
        "닉네임": nickname,
        "날짜": date,
        "본문": content,
        "댓글": comment_text
    }

# ✅ 로그인 처리
if not os.path.exists(COOKIE_PATH):
    manual_login_and_save_cookie()
else:
    load_cookie_and_login()

time.sleep(3)

# ✅ 수집 설정
current_page = 1
start_collect_page =1  # 변경 가능

# ✅ 시작 페이지까지 넘기기만 수행
while current_page < start_collect_page:
    try:
        pagination_buttons = driver.find_elements(By.CSS_SELECTOR, "#cafe_content > div.SearchBoxLayout.type_bottom > div.Pagination > button")
        clicked = False
        for btn in pagination_buttons:
            text = btn.text.strip()
            if text.isdigit() and int(text) == current_page + 1:
                current_page += 1
                driver.execute_script("arguments[0].click();", btn)
                print(f"🔁 {current_page}페이지로 이동 (수집 안함)")
                time.sleep(5)
                clicked = True
                break
        if not clicked:
            try:
                next_block_btn = driver.find_element(By.CSS_SELECTOR, "#cafe_content > div.SearchBoxLayout.type_bottom > div.Pagination > button.btn.type_next")
                if next_block_btn.is_enabled():
                    driver.execute_script("arguments[0].click();", next_block_btn)
                    print("➡️ 다음 페이지 그룹으로 이동")
                    time.sleep(5)
                else:
                    print("⛔ 다음 페이지 없음. 종료합니다.")
                    break
            except:
                print("⛔ 다음 버튼도 없음. 종료합니다.")
                break
    except Exception as e:
        print("❌ 시작 페이지 넘기기 실패:", e)
        break

# ✅ 본격적인 수집 루프
while True:
    time.sleep(3)
    collected_data = []
    article_links = get_article_elements()
    total_articles = len(article_links)
    print(f"\n📄 [{current_page}페이지] 게시글 수: {total_articles}")

    for index in range(total_articles):
        article_links = get_article_elements()
        if index >= len(article_links):
            break
        article = article_links[index]
        try:
            driver.execute_script("arguments[0].click();", article)
            time.sleep(3)
            driver.switch_to.frame("cafe_main")
            data = extract_post_data()
            collected_data.append(data)
            print(f"\r✅ {current_page}페이지 - {index + 1}/{total_articles} 수집 중...", end="", flush=True)
        except Exception as e:
            print(f"\n❌ {index + 1} 실패: {e}")
        finally:
            driver.switch_to.default_content()
            driver.back()
            time.sleep(3)

    # ✅ 페이지 수집 결과 저장
    df = pd.DataFrame(collected_data)
    if current_page == 1:
        df.to_csv("네이버카페시험키워드.csv", index=False, encoding="utf-8-sig", mode='w', header=True)
    else:
        df.to_csv("네이버카페시험키워드.csv", index=False, encoding="utf-8-sig", mode='a', header=False)
    print()  # 줄바꿈
    print(f"📁 {current_page}페이지 저장 완료")

    collected_data = []
    gc.collect()

    # ✅ 다음 페이지 이동
    try:
        pagination_buttons = driver.find_elements(By.CSS_SELECTOR, "#cafe_content > div.SearchBoxLayout.type_bottom > div.Pagination > button")
        page_clicked = False
        for btn in pagination_buttons:
            text = btn.text.strip()
            if text.isdigit() and int(text) == current_page + 1:
                current_page += 1
                driver.execute_script("arguments[0].click();", btn)
                print(f"➡️ {current_page}페이지로 이동")
                time.sleep(5)
                page_clicked = True
                break
        if not page_clicked:
            try:
                next_block_btn = driver.find_element(By.CSS_SELECTOR, "#cafe_content > div.SearchBoxLayout.type_bottom > div.Pagination > button.btn.type_next")
                if next_block_btn.is_enabled():
                    driver.execute_script("arguments[0].click();", next_block_btn)
                    current_page += 1
                    print("➡️ 다음 페이지 그룹으로 이동")
                    time.sleep(5)
                else:
                    print("⛔ 다음 페이지 없음. 종료합니다.")
                    break
            except:
                print("⛔ 다음 버튼도 없음. 종료합니다.")
                break
    except Exception as e:
        print("❌ 페이지 이동 실패:", e)
        break


📄 [1페이지] 게시글 수: 50
✅ 1페이지 - 3/50 수집 중...
📁 1페이지 저장 완료
➡️ 2페이지로 이동

📄 [2페이지] 게시글 수: 50
✅ 2페이지 - 1/50 수집 중...
📁 2페이지 저장 완료
⛔ 다음 버튼도 없음. 종료합니다.
